## [1] 논문 요약 기능

In [ ]:
!pip install transformers torch PyMuPDF --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 54.6 MB/s eta 0:00:00


In [ ]:
import re
import fitz
import textwrap
from transformers import pipeline
from pathlib import Path

In [ ]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,?!]', '', text)
    return text.strip()

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

def extract_sections_by_user_input(text, section_names):
    sections = {}
    for idx, name in enumerate(section_names):
        current = re.escape(name)
        if idx < len(section_names) - 1:
            next_name = re.escape(section_names[idx + 1])
            pattern = rf'(?i)(?:\d+\.\s*)?{current}\b[\s\S]+?(?=(?:\d+\.\s*)?{next_name}\b)'
        else:
            pattern = rf'(?i)(?:\d+\.\s*)?{current}\b[\s\S]+'
        match = re.search(pattern, text)
        if match:
            sections[name] = match.group(0).strip()
    return sections

def summarize_long_text(text, summarizer, max_chunk_len=1024, max_chunks=3):
    chunks = textwrap.wrap(text, width=max_chunk_len)
    summaries = []
    for chunk in chunks[:max_chunks]:
        try:
            summary = summarizer(chunk, max_length=80, min_length=20, do_sample=False)[0]['summary_text']
            summaries.append(summary)
        except Exception as e:
            summaries.append("[요약 실패] " + str(e))
    return ' '.join(summaries)

def pretty_print_summary(summary_text):
    sentences = re.split(r'(?<=[.!?]) +', summary_text.strip())
    for s in sentences:
        print(s.strip())

# model
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

Device set to use cuda:0


In [ ]:
# User input
input_sections = input("Enter the exact section name you want to summmarize, separated by ',' (ex: Abstract, Introduction, Methods): ")
section_names = [s.strip().lower() for s in input_sections.split(",")]

# Target paper
pdf_path = "/content/drive/MyDrive/Colab Notebooks/papers/Project Report Predicting Used Car Price.pdf"
text = extract_text_from_pdf(pdf_path).lower()

# Extract section
sections = extract_sections_by_user_input(text, section_names)

if not sections:
    print("Cannot find section name in the paper. Check out the misspelling or the exact format.")
else:
    summary_results = {}
    for sec, content in sections.items():
        if len(content.strip()) < 50:
            summary_results[sec] = "[Content is too short to summarize]"
            continue
        summary_results[sec] = summarize_long_text(content, summarizer)

    # Print summary
    for sec, summary in summary_results.items():
        print(f"\n[Section: {sec.upper()} summary]")
        pretty_print_summary(summary)

Enter the exact section name you want to summmarize, separated by ',' (ex: Abstract, Introduction, Methods): related work

[Section: RELATED WORK summary]
An alternative to directed graphical models with latent variables are undirected graphical models such as restricted boltzmann machines [27, 16] and their numerous variants .
mixing poses a signiﬁcant problem for learning algorithms that rely on mcmc [3, 5].
Some models such as denoising auto-encoders [30] and contractive autoencoders have learning rules similar to score matching applied to rbms .
in nce, as in this work, a discriminative training criterion is employed to ﬁt a generative model .
Recent work in this area includes the generative stochastic network (gsn) framework [5], which extends generalized denoising auto-encoders [4] Some techniques do not involve deﬁning a probability distribution explicitly .


## [2] 쿼리 기반 유사 논문 검색

In [ ]:
!pip install arxiv sentence-transformers faiss-cpu --quiet

In [ ]:
import arxiv
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from typing import List, Dict

In [ ]:
class ArxivPaperSearch:
    def __init__(self):
        self.model = SentenceTransformer('BAAI/bge-large-en-v1.5')
        self.index = None
        self.paper_data = []

    def fetch_papers(self, category: str = "cs.LG", max_results: int = 500) -> List[Dict]:
        """arXiv API를 사용하여 논문 데이터 수집"""
        search = arxiv.Search(
            query=f'cat:{category}',
            max_results=max_results,
            sort_by=arxiv.SortCriterion.SubmittedDate
        )

        papers = []
        for paper in arxiv.Client().results(search):
            papers.append({
                "title": paper.title,
                "abstract": paper.summary.replace('\n', ' '),
                "authors": [a.name for a in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "doi": paper.entry_id,
                "pdf_url": next(link.href for link in paper.links if link.title == "pdf")
            })
        return papers

    def build_index(self, papers: List[Dict]) -> None:
        """FAISS 인덱스 구축"""
        abstracts = [f"{p['title']} {p['abstract']}" for p in papers] # 제목 + 초록을 연결(concatenate)하여 사용
        embeddings = self.model.encode(abstracts, normalize_embeddings=True)

        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)
        self.index.add(embeddings)
        self.paper_data = papers

    def search_similar(self, query: str, top_k: int = 10) -> List[Dict]:
        """유사 논문 검색"""
        query_embedding = self.model.encode([query], normalize_embeddings=True)
        distances, indices = self.index.search(query_embedding, top_k)

        results = []
        for idx, score in zip(indices[0], distances[0]):
            if score > 0.3:  # 유사도 임계값 설정
                paper = self.paper_data[idx]
                paper['similarity'] = float(score)
                results.append(paper)
        return sorted(results, key=lambda x: x['similarity'], reverse=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Query: Used car price prediction using machine learning techniques
검색 결과 (10건):

1. Reinforcement Learning for Stock Transactions
   유사도: 0.627
   요약: Much research has been done to analyze the stock market. After all, if one can determine a pattern in the chaotic frenzy of transactions, then they co...
   PDF: http://arxiv.org/pdf/2505.16099v1

2. Uncertainty Quantification in SVM prediction
   유사도: 0.620
   요약: This paper explores Uncertainty Quantification (UQ) in SVM predictions, particularly for regression and forecasting tasks. Unlike the Neural Network, ...
   PDF: http://arxiv.org/pdf/2505.15429v1

3. Interpretable Machine Learning for Macro Alpha: A News Sentiment Case Study
   유사도: 0.607
   요약: This study introduces an interpretable machine learning (ML) framework to extract macroeconomic alpha from global news sentiment. We process the Globa...
   PDF: http://arxiv.org/pdf/2505.16136v1

4. A system identification approach to clustering vector autoregressive time series
   유사

| 코드      | 분야 이름                          | 설명                         |
| ------- | ------------------------------ | -------------------------- |
| `cs.AI` | Artificial Intelligence        | 인공지능, 지식 표현, 계획, 전문가 시스템 등 |
| `cs.LG` | Machine Learning               | 지도/비지도/강화 학습, 딥러닝 포함       |
| `cs.CL` | Computation and Language       | 자연어 처리 (NLP), 언어 모델        |
| `cs.CV` | Computer Vision                | 이미지 분류, 객체 탐지, 영상 처리 등     |
| `cs.IR` | Information Retrieval          | 검색 시스템, 추천 시스템             |
| `cs.DS` | Data Structures and Algorithms | 알고리즘 설계, 복잡도 이론            |
| `cs.CR` | Cryptography and Security      | 암호학, 보안 프로토콜, 시스템 보안       |
| `cs.RO` | Robotics                       | 로봇공학, 자율주행, 로보틱 모션         |
| `cs.HC` | Human-Computer Interaction     | 사용자 인터페이스, UX              |
| `cs.SE` | Software Engineering           | 소프트웨어 개발, 테스트, 유지보수        |
| `stat.ML` | Machine Learning | 통계적 관점의 머신러닝               |
| `stat.AP` | Applications     | 다양한 분야의 통계 응용              |
| `stat.TH` | Theory           | 수리적 통계 이론                  |
| `stat.CO` | Computation      | 통계적 계산 방법, MCMC, EM 알고리즘 등 |
| `math.OC` | Optimization and Control | 최적화 이론, 제어 이론 |
| `math.ST` | Statistics               | 수학적 통계        |
| `math.PR` | Probability              | 확률 이론         |

In [ ]:
# 사용 예시
if __name__ == "__main__":
    search_system = ArxivPaperSearch()

    # 1. arXiv에서 머신러닝 분야 논문 500편 수집
    papers = search_system.fetch_papers(category="cs.LG", max_results=500)

    # 2. 임베딩 인덱스 구축
    search_system.build_index(papers)

    # 3. 유사 논문 검색 실행
    # query = "Used car price prediction using machine learning techniques"
    query = input("Query: ")
    results = search_system.search_similar(query)

    # 4. 결과 출력
    print(f"검색 결과 ({len(results)}건):")
    for i, paper in enumerate(results[:5]):
        print(f"\n{i+1}. {paper['title']}")
        print(f"   유사도: {paper['similarity']:.3f}")
        print(f"   요약: {paper['abstract'][:150]}...")
        print(f"   PDF: {paper['pdf_url']}")

Query: random forest
검색 결과 (10건):

1. Margin-aware Fuzzy Rough Feature Selection: Bridging Uncertainty Characterization and Pattern Classification
   유사도: 0.676
   요약: Fuzzy rough feature selection (FRFS) is an effective means of addressing the curse of dimensionality in high-dimensional data. By removing redundant a...
   PDF: http://arxiv.org/pdf/2505.15250v1

2. InTreeger: An End-to-End Framework for Integer-Only Decision Tree Inference
   유사도: 0.666
   요약: Integer quantization has emerged as a critical technique to facilitate deployment on resource-constrained devices. Although they do reduce the complex...
   PDF: http://arxiv.org/pdf/2505.15391v1

3. SplitWise Regression: Stepwise Modeling with Adaptive Dummy Encoding
   유사도: 0.664
   요약: Capturing nonlinear relationships without sacrificing interpretability remains a persistent challenge in regression modeling. We introduce SplitWise, ...
   PDF: http://arxiv.org/pdf/2505.15423v1

4. Fairness under Competition
   유사도: 0.663
   요

## [3] 논문 기반 Q & A

In [ ]:
from transformers import pipeline
import fitz

# PDF에서 텍스트 추출
doc = fitz.open("/content/drive/MyDrive/Colab Notebooks/papers/Project Report Predicting Used Car Price.pdf")
full_text = ""
for page in doc:
    full_text += page.get_text()

# 슬라이딩 윈도우로 나누기 (예: 1000자 단위, 500자 겹치기)
window_size = 1000
stride = 500
contexts = []
for i in range(0, len(full_text), stride):
    contexts.append(full_text[i:i+window_size])

# QA 파이프라인
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# 각 구간별로 QA 수행
question = "What data was used?"
answers = []
for ctx in contexts:
    result = qa_pipeline(question=question, context=ctx)
    if result["score"] > 0.2:  # confidence 임계값 설정 (필요시 조정)
        answers.append((result["answer"], result["score"]))

# confidence가 높은 순으로 정렬
answers.sort(key=lambda x: x[1], reverse=True)

# 최상위 답변 출력
if answers:
    print("Best Answer:", answers[0][0])
    print("Confidence:", answers[0][1])
else:
    print("No answer found.")